**Alumno:** César Emilio García Ávalos

**Actividad:** M5_AI1_Modelo lineal Gaussiano_modelos espaciales

In [ ]:
install.packages(c("sf", "spdep", "dplyr", "ggplot2","osmdata","geosphere","MASS","earth","spatialreg","caret","spgwr"))
library(spgwr)
library(caret)
library(caret)
library(earth)
library(spdep)
library(MASS)
library(geosphere)
library(osmdata)
library(sf)
library(dplyr)
library(ggplot2)

In [ ]:
# Cargar los datos
tabla <- read.csv("https://raw.githubusercontent.com/cesargar1507/DatasetsUB/main/table_5.05.csv", sep = ",", header = TRUE)

In [ ]:
summary(tabla)

In [ ]:
unique(tabla$room_type)
unique(tabla$instant_bookable)

In [ ]:
sum(is.na(tabla))

In [ ]:
# Añadir una pequeña perturbación aleatoria a la latitud y longitud
set.seed(123)
tabla$latitude <- tabla$latitude + runif(nrow(tabla)) / 1000
tabla$longitude <- tabla$longitude + runif(nrow(tabla)) / 1000

# Seleccionar un subconjunto de datos si hay más de 2000 filas
if (nrow(tabla) > 2000) {
  set.seed(1)
  tabla <- tabla[sample(nrow(tabla), 2000), ]
}
pisos<- tabla

In [ ]:
coords <- cbind(tabla$longitude, tabla$latitude)
nb <- knn2nb(knearneigh(coords, k = 10))

In [ ]:
# Calcular la matriz de pesos espaciales
listw <- nb2listw(nb, style = "W")
moran_global <- moran.test(x = tabla$price, listw = listw)
moran.plot(x = tabla$price, listw = listw, main = "Gráfico de Moran")
moran_global

In [ ]:
imoranlocal<-as.data.frame(localmoran(x = tabla$price, listw = listw))
tabla$registo<-1
#pl_pt(tabla,color2 = imoranlocal$Z.Ii,size2 =tabla$registo ,dd = 6)

In [ ]:
pisos <- dplyr::select(pisos,-room_type)

In [ ]:
modelo_lineal <- glm(price ~ ., data = pisos, family = gaussian)
# Resumen del modelo
summary(modelo_lineal)

In [ ]:
moran.test(x=modelo_lineal$residuals, listw = listw)
moran.plot(x=modelo_lineal$residuals, listw = listw)

In [ ]:
# Definir el área de Madrid
bbox <- getbb("Madrid, Spain")

# Consultar OSM para obtener ubicaciones de oficinas bancarias
bancos <- opq(bbox = bbox) %>%
  add_osm_feature(key = 'amenity', value = 'bank') %>%
  osmdata_sf()

# Extraer las coordenadas de los bancos
bancos_coords <- bancos$osm_points %>% st_coordinates()

In [ ]:
# Crear una función para calcular la distancia mínima
calcular_distancia_minima <- function(lat, lon, bancos_coords) {
  distancias <- distm(c(lon, lat), bancos_coords)
  min(distancias)
}

pisosBank<-pisos
# Calcular la distancia mínima para cada observación en tus datos
pisosBank$distancia_bancos <- mapply(calcular_distancia_minima, pisos$latitude, pisos$longitude, MoreArgs = list(bancos_coords = bancos_coords))

In [ ]:
modelo_lineal_completo <- glm(price~., data = pisosBank, family = gaussian)
# Resumen del modelo
summary(modelo_lineal_completo)

In [ ]:
moran.test(x=modelo_lineal_completo$residuals, listw = listw)
moran.plot(x=modelo_lineal_completo$residuals, listw = listw)

In [ ]:
paste("Residuos del modelo lineal", sum((modelo_lineal$residuals)**2))
paste("Residuos del modelo lineal con distancia a bancos", sum((modelo_lineal_completo$residuals)**2))

In [ ]:
backward <- stepAIC(modelo_lineal_completo,trace=FALSE,direction="backward")
backward$anova
summary(backward)

In [ ]:
formula <- as.formula("price ~ longitude + minimum_nights + number_of_reviews +
    review_scores_value + calculated_host_listings_count + reviews_per_month +
    Distancia_Centro + Distancia_Sur + logprice")

In [ ]:
modelo_sar <- lagsarlm(formula = formula, data= pisosBank, listw = listw)
summary(modelo_sar)

In [ ]:
# Ajustar el modelo SEM
modelo_sem <- errorsarlm(formula = formula, data = pisosBank, listw = listw)
summary(modelo_sem)

In [ ]:
paste("Residuos del modelo lineal", sum((modelo_lineal$residuals)**2))
paste("Residuos del modelo lineal con distancia a bancos", sum((modelo_lineal_completo$residuals)**2))
paste("Residuos del modelo SAR", sum((modelo_sar$residuals)**2))
paste("Residuos del modelo SEM", sum((modelo_sem$residuals)**2))

In [ ]:
moran.test(x=modelo_sar$residuals, listw = listw)
moran.test(x=modelo_sem$residuals, listw = listw)

In [ ]:
# Definir el número de folds para la validación cruzada
num_folds <- 5

# Inicializar vectores para almacenar resultados de rendimiento
mse_values <- numeric(num_folds)
mae_values <- numeric(num_folds)
r_squared_values <- numeric(num_folds)

# Realizar la validación cruzada
for (fold in 1:num_folds) {
  # Generar índices de división aleatorios
  fold_indices <- createDataPartition(pisosBank$logprice, times = 1, p = 0.7, list = FALSE)

  # Subconjuntos de entrenamiento y prueba
  train_data <- pisosBank[fold_indices, ]
  test_data <- pisosBank[-fold_indices, ]

  # Crear matriz de pesos espaciales solo con los datos de entrenamiento de este fold
 coords_train <- cbind(train_data$longitude, train_data$latitude)
 nb_train <- knn2nb(knearneigh(coords_train, k = 10))
 train_listw <- nb2listw(nb_train, style = "W")

 # Crear matriz de pesos espaciales solo con los datos de prueba de este fold
 coords_test <- cbind(test_data$longitude, test_data$latitude)
 nb_test <- knn2nb(knearneigh(coords_test, k = 10))
 test_listw <- nb2listw(nb_test, style = "W")
 row.names(test_data) <- test_data$region.id

  # Ajustar el modelo SAR en el subconjunto de entrenamiento
  modelo_sar_fold <- lagsarlm(logprice ~ ., data = train_data, listw = train_listw)

  # Realizar predicciones en el conjunto de prueba
  predicciones <- predict(modelo_sar_fold, newdata = test_data, listw = test_listw)

  # Calcular métricas de rendimiento
  mse_values[fold] <- mean((test_data$logprice - predicciones)^2)
  mae_values[fold] <- mean(abs(test_data$logprice - predicciones))
  r_squared_values[fold] <- cor(test_data$logprice, predicciones)^2
}

# Calcular el promedio de las métricas de rendimiento
mean_mse <- mean(mse_values)
mean_mae <- mean(mae_values)
mean_r_squared <- mean(r_squared_values)

# Imprimir resultados
cat("MSE promedio:", mean_mse, "\n")
cat("MAE promedio:", mean_mae, "\n")
cat("R cuadrado promedio:", mean_r_squared, "\n")


In [ ]:
# Ajustar modelo GWR y seleccionar ancho de banda automáticamente
tabla$residuos<-modelo_sem$residuals
puntos_sp<-tabla
coordinates(puntos_sp)<- c("longitude","latitude")
proj4string(puntos_sp) <- CRS("+proj=longlat +datum=WGS84")
#Obtenemos el mejor BW
bw <- gwr.sel(residuos~1, data=puntos_sp)

paste("El mejor ancho de banda es:",bw)

In [ ]:
g <- gwr(residuos~1, data=puntos_sp, bandwidth=bw)